In [1]:
# add repo root to beginning of sys path to allow other modules to be found by python import machinery
import sys 
import os
import boto3
from pathlib import Path
sys.path
ROOT_PATH = os.path.abspath('..')
sys.path.insert(0, ROOT_PATH)
RESULTS_DIR = os.path.join(str(Path(ROOT_PATH).parents[1]), 'datasets/forecasting/results')


In [2]:
from create_predictor_forecast_jobs import *
from common import *
from teardown_resources import *


In [15]:
# forecast horizon 
FORECAST_LENGTH = 60
# forecast time unit granularity
DATASET_FREQUENCY = "D"
TIMESTAMP_FORMAT = "yyyy-MM-dd"
dataset_name = "manning_csv_full"
predictor_name = f"automl_3bt_60h_explain"
explain_name='explain_manning'
explain_export_name = 'explain_manning_full_dataset_automl'
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
ts_schema ={
   "Attributes":[
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      }

   ]
}



role_name = "AmazonForecast-ExecutionRole-1645141603603"
iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']

#### Create predictor

Need to use auto-predictor as explainability is only supported for predictors created in auto ml mode.

*Note This will take a while  to create the resource and train ~30 mins-1hr ! 

In [12]:
dataset_group_arn = "arn:aws:forecast:us-east-1:376337229415:dataset-group/manning"
create_predictor_response , predictor_arn = train_aws_forecast_model(predictor_name, FORECAST_LENGTH, DATASET_FREQUENCY, dataset_group_arn, auto_ml=True, explain=True, holidays_code="US")



In [6]:
check_job_status(predictor_arn, job_type="training", auto_ml=True, wait_time=1200)

Import job still in progress. Job status CREATE_IN_PROGRESS
Import job still in progress. Job status CREATE_IN_PROGRESS
Import job still in progress. Job status ACTIVE
Training job complete with job status ACTIVE


#### Create Explainability and export to S3

In [14]:

#response = create_explainability_job(explain_name, predictor_arn)
#explain_arn = respone['ExplainabilityArn']
explain_arn = forecast.list_explainabilities()['Explainabilities'][0]['ExplainabilityArn']

In [18]:
s3_export_path =  "s3://aws-forecast-demo-examples/aws_forecast_export_jobs/"

response = forecast.create_explainability_export(
    ExplainabilityExportName=explain_export_name,
    ExplainabilityArn=explain_arn,
    Destination={
        'S3Config': {
            'Path': s3_export_path,
            'RoleArn': role_arn
        }
    }
)


#### Delete all resources

passing in predictor arn will delete the predictor resource and all child explainability resources as per the 
parent << child resource hierarchies: Dataset << Dataset Group << Predictor << Forecast
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/forecast.html#ForecastService.Client.delete_resource_tree

In [3]:
response = delete_resource_tree(predictor_arn)
response

{'ResponseMetadata': {'RequestId': 'c80fdc99-b39f-4ac4-8d73-1b0ded3001db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 17 May 2022 03:36:34 GMT',
   'x-amzn-requestid': 'c80fdc99-b39f-4ac4-8d73-1b0ded3001db',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}